## 📌 Step 5: Measure Angle Between Joints (Shoulder, Elbow, Wrist)

#### Now that we can track the shoulders, let’s calculate the angle between joints to evaluate form. This is useful for checking exercise posture like bicep curls, squats, or push-ups.
## 🔢 How to Calculate the Angle?

We use the cosine rule:
θ=cos⁡−1(b2+c2−a22bc)
θ=cos−1(2bcb2+c2−a2​)

where:

    a, b, c are the distances between points.
    θ is the joint angle (in degrees).

We'll calculate elbow flexion angle using:

    Shoulder
    Elbow
    Wrist

In [ ]:
import cv2 
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Start capturing video
cap = cv2.VideoCapture(0)

def calculate_angles(a, b, c):
    """Calculate angle between three points"""
    a = np.array(a)  # First point
    b = np.array(b)  # Mid Point (joint)
    c = np.array(c)  # Last point

    # Calculate vectors
    ba = a - b
    bc = c - b

    # Compute cosine of the angle
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))

    # Convert to degrees
    angle = np.degrees(np.arccos(cosine_angle))

    return angle

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to grab frame")
        break  # Exit loop if no frame is received

    frame_height, frame_width, _ = frame.shape

    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process pose detection
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        landmarks = results.pose_landmarks.landmark

        try:
            # ✅ LEFT ARM
            left_shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * frame_width),
                             int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * frame_height))

            left_elbow = (int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x * frame_width),
                          int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y * frame_height))

            left_wrist = (int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x * frame_width),
                          int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * frame_height))

            left_angle = calculate_angles(left_shoulder, left_elbow, left_wrist)

            # ✅ RIGHT ARM
            right_shoulder = (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * frame_width),
                              int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * frame_height))

            right_elbow = (int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x * frame_width),
                           int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y * frame_height))

            right_wrist = (int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x * frame_width),
                           int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y * frame_height))

            right_angle = calculate_angles(right_shoulder, right_elbow, right_wrist)

            # Display angles on screen
            cv2.putText(frame, f"L Angle: {int(left_angle)}°", (left_elbow[0] - 50, left_elbow[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

            cv2.putText(frame, f"R Angle: {int(right_angle)}°", (right_elbow[0] - 50, right_elbow[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

            # Draw circles on key joints
            cv2.circle(frame, left_shoulder, 5, (255, 0, 0), -1)
            cv2.circle(frame, left_elbow, 5, (0, 255, 0), -1)
            cv2.circle(frame, left_wrist, 5, (0, 0, 255), -1)

            cv2.circle(frame, right_shoulder, 5, (255, 0, 0), -1)
            cv2.circle(frame, right_elbow, 5, (0, 255, 0), -1)
            cv2.circle(frame, right_wrist, 5, (0, 0, 255), -1)

        except IndexError:
            print("Landmarks not detected properly, skipping frame.")

    # Show output
    cv2.imshow("Pose Detection with Angle Calculation", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ✅ FIXED: Properly release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import streamlit as st
import cv2
import mediapipe as mp
import numpy as np
import tempfile
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Pose Detection & Rep Counting
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
    angles_list = []
    skeleton_points = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            landmarks = results.pose_landmarks.landmark
            
            shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * frame_width),
                        int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * frame_height))
            elbow = (int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x * frame_width),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y * frame_height))
            wrist = (int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x * frame_width),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * frame_height))
            
            angle = calculate_angle(shoulder, elbow, wrist)
            angles_list.append(angle)
            
            skeleton_points.append([
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, 
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y, 
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].z
            ])
            
            cv2.putText(frame, f'Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
            cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
            cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        
        out.write(frame)
    
    cap.release()
    out.release()
    
    reps = count_reps(angles_list)
    return 'output.mp4', reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

# Streamlit UI
st.title("AI Fitness Coach - Pose Estimation, Rep Counter & 3D Visualization")

uploaded_file = st.file_uploader("Upload an exercise video", type=["mp4", "mov", "avi"])
view_option = st.radio("Select View Mode:", ["Skeleton (Lines)", "3D Muscle Body"])

if uploaded_file is not None:
    with tempfile.NamedTemporaryFile(delete=False) as temp_file:
        temp_file.write(uploaded_file.read())
        video_path = temp_file.name
    
    st.video(video_path)
    st.write("Processing video...")
    
    output_video_path, rep_count, skeleton_points = process_video(video_path)
    
    st.video(output_video_path)
    st.success(f"Processing Complete! Total Reps Counted: {rep_count}")
    
    if view_option == "Skeleton (Lines)":
        st.plotly_chart(generate_3d_pose(skeleton_points))
    else:
        st.write("3D Muscle Body Visualization (Coming Soon)")


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    angles_list = []
    skeleton_points = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            landmarks = results.pose_landmarks.landmark
            
            joints = {
                'left_shoulder': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                'right_shoulder': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
            }
            
            for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                            int(landmarks[shoulder_idx].y * frame_height))
                elbow = (int(landmarks[elbow_idx].x * frame_width),
                         int(landmarks[elbow_idx].y * frame_height))
                wrist = (int(landmarks[wrist_idx].x * frame_width),
                         int(landmarks[wrist_idx].y * frame_height))
                
                angle = calculate_angle(shoulder, elbow, wrist)
                angles_list.append(angle)
                
                skeleton_points.append([
                    landmarks[shoulder_idx].x, 
                    landmarks[shoulder_idx].y, 
                    landmarks[shoulder_idx].z
                ])
                
                cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    reps = count_reps(angles_list)
    return reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

if __name__ == "__main__":
    live_tracking()


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    angles_list = []
    skeleton_points = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left_shoulder': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right_shoulder': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                int(landmarks[shoulder_idx].y * frame_height))
                    elbow = (int(landmarks[elbow_idx].x * frame_width),
                             int(landmarks[elbow_idx].y * frame_height))
                    wrist = (int(landmarks[wrist_idx].x * frame_width),
                             int(landmarks[wrist_idx].y * frame_height))
                    
                    angle = calculate_angle(shoulder, elbow, wrist)
                    angles_list.append(angle)
                    
                    skeleton_points.append([
                        landmarks[shoulder_idx].x, 
                        landmarks[shoulder_idx].y, 
                        landmarks[shoulder_idx].z
                    ])
                    
                    cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                    cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                    cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            print(f"Error processing landmarks: {e}")
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    reps = count_reps(angles_list)
    return reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

if __name__ == "__main__":
    live_tracking()


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    angles_list = []
    skeleton_points = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left_shoulder': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right_shoulder': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                int(landmarks[shoulder_idx].y * frame_height))
                    elbow = (int(landmarks[elbow_idx].x * frame_width),
                             int(landmarks[elbow_idx].y * frame_height))
                    wrist = (int(landmarks[wrist_idx].x * frame_width),
                             int(landmarks[wrist_idx].y * frame_height))
                    
                    angle = calculate_angle(shoulder, elbow, wrist)
                    angles_list.append(angle)
                    
                    skeleton_points.append([
                        landmarks[shoulder_idx].x, 
                        landmarks[shoulder_idx].y, 
                        landmarks[shoulder_idx].z
                    ])
                    
                    cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                    cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                    cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            print(f"Error processing landmarks: {e}")
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    reps = count_reps(angles_list)
    return reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

if __name__ == "__main__":
    live_tracking()


In [5]:
import cv2
import mediapipe as mp
import numpy as np
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    angles_list = []
    skeleton_points = []
    peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left_shoulder': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right_shoulder': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                int(landmarks[shoulder_idx].y * frame_height))
                    elbow = (int(landmarks[elbow_idx].x * frame_width),
                             int(landmarks[elbow_idx].y * frame_height))
                    wrist = (int(landmarks[wrist_idx].x * frame_width),
                             int(landmarks[wrist_idx].y * frame_height))
                    
                    angle = calculate_angle(shoulder, elbow, wrist)
                    angles_list.append(angle)
                    
                    skeleton_points.append([
                        landmarks[shoulder_idx].x, 
                        landmarks[shoulder_idx].y, 
                        landmarks[shoulder_idx].z
                    ])
                    
                    cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                    cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                    cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            print(f"Error processing landmarks: {e}")
        
        # Display muscle concentration bar
        muscle_concentration = np.mean(angles_list) if angles_list else 0
        peak_concentration = max(peak_concentration, muscle_concentration)
        bar_length = int(muscle_concentration / 180 * 200)  # Scale for visualization
        cv2.rectangle(frame, (50, 50), (50 + bar_length, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Muscle Concentration: {int(muscle_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Display peak concentration
        cv2.putText(frame, f'Peak Concentration: {int(peak_concentration)}%', (50, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    reps = count_reps(angles_list)
    return reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

if __name__ == "__main__":
    live_tracking()


KeyboardInterrupt: 

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    angles_list = []
    skeleton_points = []
    peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left_shoulder': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right_shoulder': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                int(landmarks[shoulder_idx].y * frame_height))
                    elbow = (int(landmarks[elbow_idx].x * frame_width),
                             int(landmarks[elbow_idx].y * frame_height))
                    wrist = (int(landmarks[wrist_idx].x * frame_width),
                             int(landmarks[wrist_idx].y * frame_height))
                    
                    angle = calculate_angle(shoulder, elbow, wrist)
                    angles_list.append(angle)
                    
                    skeleton_points.append([
                        landmarks[shoulder_idx].x, 
                        landmarks[shoulder_idx].y, 
                        landmarks[shoulder_idx].z
                    ])
                    
                    cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                    cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                    cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            print(f"Error processing landmarks: {e}")
        
        # Display muscle concentration bar
        muscle_concentration = angles_list[-1] if angles_list else 0  # Use the latest angle for faster response
        peak_concentration = max(peak_concentration, muscle_concentration)
        bar_length = int(muscle_concentration / 180 * 200)  # Scale for visualization
        cv2.rectangle(frame, (50, 50), (50 + bar_length, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Muscle Concentration: {int(muscle_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Display peak concentration
        cv2.putText(frame, f'Peak Concentration: {int(peak_concentration)}%', (50, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    reps = count_reps(angles_list)
    return reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

if __name__ == "__main__":
    live_tracking()


In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
from flask import Flask, Response, request, jsonify
from flask_cors import CORS

# Optimize threading
os.environ["OMP_NUM_THREADS"] = "8"

# Flask app
app = Flask(_name_)
CORS(app)

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Exercise count tracking
exercise_count = 0
prev_position = None

# Function to calculate joint angles
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

# Process video stream
def generate_frames():
    global exercise_count, prev_position
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Extract key points
            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y]

            # Calculate angles
            elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

            # Push-up detection
            if elbow_angle < 40:  
                if prev_position == "up":
                    exercise_count += 1
                prev_position = "down"
            elif elbow_angle > 160:
                prev_position = "up"

            # Crunches detection
            if knee_angle < 90:
                if prev_position == "extended":
                    exercise_count += 1
                prev_position = "crunched"
            elif knee_angle > 150:
                prev_position = "extended"

        # Convert frame to JPEG
        _, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

    cap.release()
    cv2.destroyAllWindows()

# API for video feed
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

# API for exercise count
@app.route('/get_count', methods=['GET'])
def get_count():
    return jsonify({"count": exercise_count})

# Test route
@app.route('/')
def index():
    return "Backend is running!"

if _name_ == "_main_":
    app.run(host="0.0.0.0", port=5000, debug=True)